In [1]:
# 1. Librerías que necesitamos
import pandas as pd
from elasticsearch import Elasticsearch  # Cliente oficial de ES
from datetime import datetime
import time, os

# 2. Conectarnos a Elasticsearch (sin autenticación porque es laboratorio)
ES = Elasticsearch("http://localhost:9200")

# 3. Comprobar que ES está vivo
if ES.ping():
    print("✅ Elasticsearch responde")
else:
    raise ValueError("❌ No se pudo conectar a ES")

✅ Elasticsearch responde


In [2]:
# 4. Leer el CSV que creamos en Fase 3
CSV_PATH = "predicciones_100k.csv"
df = pd.read_csv(CSV_PATH)

# 5. Quedarnos **solo** con las anomalías (anomaly = 1)
alertas = df[df["anomaly"] == 1].copy()
print(f"Total de alertas a enviar: {len(alertas)}")

Total de alertas a enviar: 5000


In [3]:
# 6. Elasticsearch exige un campo @timestamp para los gráficos
alertas["@timestamp"] = pd.to_datetime(alertas["timestamp"]).dt.strftime("%Y-%m-%dT%H:%M:%S")

# 7. Convertir NaN a string para que no fallen los índices
alertas = alertas.fillna("null")

In [4]:
# 8. Índice dinámico con la fecha de hoy
index_name = f"alertas-anomalia-{datetime.now().strftime('%Y.%m.%d')}"

# 9. Enviar cada alerta
for _, fila in alertas.iterrows():
    doc = fila.to_dict()
    resp = ES.index(index=index_name, body=doc)
    # Imprimimos solo cada 500 para no llenar pantalla
    if _ % 500 == 0:
        print(f"  Enviado doc {_} -> {resp['result']}")

print(f"🟢 Terminado. Índice: {index_name}")

  Enviado doc 2000 -> created
  Enviado doc 5500 -> created
  Enviado doc 14500 -> created
  Enviado doc 43000 -> created
  Enviado doc 49500 -> created
  Enviado doc 52000 -> created
  Enviado doc 52500 -> created
  Enviado doc 92500 -> created
  Enviado doc 95000 -> created
  Enviado doc 99000 -> created
🟢 Terminado. Índice: alertas-anomalia-2025.11.20
